This notebook contains a simulation and code for testing and developing a VAE for fitting models across multiple subjects

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
%matplotlib qt

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

## Define helper functions here

In [14]:
def gen_couplings(neuron_ctrs, mode_ctr, mode_std, mode_ctr_mean, mode_noise_std):
    """ Generates ground truth couplings. """
    weighted_distance_from_ctrs = np.sum(((neuron_ctrs - mode_ctr)/mode_std)**2, axis=1)
    coupling_means = mode_ctr_mean*np.exp(-1*weighted_distance_from_ctrs)
    return np.random.randn(len(coupling_means))*mode_noise_std + coupling_means 

def visualize_couplings(neuron_ctrs, p_couplings, u_couplings, max_mag=1):
    """ Generates plots of couplings. """
    n_neurons = neuron_ctrs.shape[0]
    n_modes = p_couplings.shape[1]
    
    cmap = matplotlib.cm.get_cmap('PiYG')
    
    for m_i in range(n_modes):
        a = plt.subplot(n_modes, 2, 2*m_i + 1)
        for n_i in range(n_neurons):
            c_vl = p_couplings[n_i, m_i]/(2*max_mag) + .5
            clr = cmap(c_vl)
            p = plt.plot(neuron_ctrs[n_i,0], neuron_ctrs[n_i,1], 'ko', 
                        alpha=.5, markerfacecolor=clr)
        a.set_xticklabels([])
        a.set_yticklabels([])
        if m_i == 0:
            plt.title('P Couplings')
        
        a = plt.subplot(n_modes, 2, 2*m_i + 2)
        for n_i in range(n_neurons):
            c_vl = u_couplings[n_i, m_i]/(2*max_mag) + .5
            clr = cmap(c_vl)
            p = plt.plot(neuron_ctrs[n_i,0], neuron_ctrs[n_i,1], 'ko', 
                         alpha=.5, markerfacecolor=clr)
        a.set_xticklabels([])
        a.set_yticklabels([])
        if m_i == 0:
            plt.title('U Couplings')

## Parameters go here

In [4]:
n_neurons_per_subject = [90, 100, 110, 80, 102] # Length implicitly defines # of subjects

n_smps_per_subject = [1000, 1100, 900, 999, 1230] # Number of (x,y) pairs for each subject

# We model distributions of couplings with bumps - where the magnitude of a coupling is a 
# function of how far that neuron is from the center of the bump + some noise 

# Here we specify the centers where couplings are heighest for each p mode
p_mode_centers = np.asarray([[.2, .2], [.5, .5], [.8, .8]]) 
# Here we specify the standard deviation of the spatial extents of couplings for each p mode
p_mode_stds = np.asarray([[.3, .3], [.3, .3], [.3, .3]]) 
# Here we define the mean value for a coupling at the center of each mode
p_mode_mean_center_coupling = [1, 1, 1]
# Here we define how much noise to add when generating couplings for each mode
p_mode_noise_stds = [.05, .05, .05]

# Parmeters for generate u mode couplings 
u_mode_centers = np.asarray([[.2, .2], [.5, .5], [.8, .8]]) # Physical centers of u modes
u_mode_stds = np.asarray([[.4, .4], [.4, .4], [.4, .4]])
u_mode_mean_center_coupling = [1, -1, 1]
u_mode_noise_stds = [.05, .05, .05]

# Range to generate neuron private noise variances from, for each subject
priv_noise_var_ranges = np.asarray([[.5, .8],
                                    [.5, .8],
                                    [.5, .8],
                                    [.5, .8],
                                    [.5, .8]])

## Generate each ground truth model and data

In [36]:
n_subjects = len(n_neurons_per_subject)
n_modes = p_mode_centers.shape[0]
subjects = [None]*n_subjects
for s_i in range(n_subjects):
    n_neurons = n_neurons_per_subject[s_i]
    n_smps = n_smps_per_subject[s_i]
    
    # Generate neuron locations uniformly in the unit square
    neuron_x = np.random.rand(n_neurons)
    neuron_y = np.random.rand(n_neurons)
    neuron_ctrs = np.stack([neuron_x, neuron_y]).transpose()
    
    # Generate couplings 
    p = np.zeros([n_neurons, n_modes])
    u = np.zeros([n_neurons, n_modes])
    for m_i in range(n_modes):
        p[:, m_i] = gen_couplings(neuron_ctrs, p_mode_centers[m_i,:], p_mode_stds[m_i,:], 
                          p_mode_mean_center_coupling[m_i], p_mode_noise_stds[m_i])
        u[:, m_i] = gen_couplings(neuron_ctrs, u_mode_centers[m_i,:], u_mode_stds[m_i,:], 
                          u_mode_mean_center_coupling[m_i], u_mode_noise_stds[m_i])
        
    # Generate x smps - these just come from a standard normal 
    x = np.random.randn(n_smps, n_neurons)
    
    # Generate means for each y smp
    l = np.matmul(x, p)
    y_mn = np.matmul(l, u.transpose())
    
    # Generate private noise variances
    priv_var_width = priv_noise_var_ranges[s_i,1] - priv_noise_var_ranges[s_i,0]
    priv_var = np.random.rand(n_neurons)*priv_var_width + priv_noise_var_ranges[s_i,0]
    
    # Generate y smps
    y_noise = np.random.randn(n_smps, n_neurons)*priv_var
    y = y_mn + y_noise
    
    # Store results
    s_dict = dict()
    s_dict['n_neurons'] = n_neurons
    s_dict['n_smps'] = n_smps
    s_dict['neuron_ctrs'] = neuron_ctrs
    s_dict['p'] = p
    s_dict['u'] = u
    s_dict['priv_var'] = priv_var
    s_dict['x'] = x
    s_dict['y'] = y
    s_dict['y_mn'] = y_mn
    
    subjects[s_i] = s_dict

## Look at ground truth data for one subject

In [37]:
vis_subj = 1 # The subject we want to visualize ground truth for
subject = subjects[vis_subj]


# Visualize couplings
plt.figure()
visualize_couplings(subject['neuron_ctrs'], 
                    subject['p'], 
                    subject['u'])

# Visualize y data
roi_inds = np.arange(0, subject['n_neurons'], 10)
plt.figure()
for i, roi_i in enumerate(roi_inds):
    plt.subplot(len(roi_inds), 1, i+1)
    plt.plot(subject['y_mn'][:, roi_i], 'b-')
    plt.plot(subject['y'][:, roi_i], 'ro', markersize=2)